In [12]:
#%matplotlib inline
import time
import warnings
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import numpy as np
import os
import pathlib
import tensorflow as tf
import yaml
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
warnings.filterwarnings('ignore')  # Suppress Matplotlib warnings

tf.get_logger().setLevel('ERROR')  # Suppress TensorFlow logging (2)

with open("config.yaml" , "r") as config_file:
    config = yaml.load(config_file , Loader=yaml.FullLoader)

def get_Image_Paths_Array(path):
    """
    @path ist ein pfad zu einem Ordner mit unterordner die Bilder Inhalten
    :returns ein Array mit den pfaden zu jedem Bild in den Unterordner
    """
    array = []
    for image in os.listdir(path):
        fullFileName = os.path.join(path, image)
        array.append(fullFileName)
    return array

PATH_TO_LABELS = config['PATH_TO_LABEL_MAP']
PATH_TO_ALL_IMAGES = config['PATH_TO_ALL_IMAGES']
PATH_TO_MODEL_DIR = config['PATH_TO_EXPORTED_MODEL']
PATH_TO_PLOTS = config['PATH_TO_PLOTS']
IMAGE_PATHS = get_Image_Paths_Array(PATH_TO_ALL_IMAGES)

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR
print('Loading model...', end='')
start_time = time.time()
# Load saved model and build the detection function
model = tf.saved_model.load(PATH_TO_SAVED_MODEL)
end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))



Loading model...Done! Took 15.282128095626831 seconds


In [13]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
start_time = time.time()
import matplotlib
matplotlib.use("pdf")
from PIL import Image
import matplotlib.pyplot as plt
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

id = 1
#detected_Images = np.unique(detected_Images)
for image_path in IMAGE_PATHS:
    print('Running inference for {}... '.format(image_path), end='')

    if '.DS_Store' in image_path:
        continue

    image_np = load_image_into_numpy_array(image_path)
    if len(image_np.shape) == 2:
        image_np = np.stack((image_np,) * 3, axis=-1)

    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...]
    time1 = time.time()
    detections = model(input_tensor)  # create detection

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    time2 = time.time()
    print("took: ", time2 - time1)
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],

        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.3,
        agnostic_mode=False)

    plt.figure()
    plt.imshow(image_np_with_detections)
    #plt.show()
    matplotlib.pyplot.savefig(PATH_TO_PLOTS + str(id))
    id = id + 1


Running inference for ./images/all_images/BA 2005-01201 b_Scan01.jpg... 

2023-01-28 18:21:10.612853: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


took:  4.193482875823975
Running inference for ./images/all_images/BA 2005-00563_Scan01.jpg... took:  0.5204000473022461
Running inference for ./images/all_images/BA 2004-12965-0584_Scan01.jpg... took:  0.2695150375366211
Running inference for ./images/all_images/BA 2004-00200_Dok01.jpg... took:  0.5183010101318359
Running inference for ./images/all_images/BA 2004-02267_Scan01.jpg... took:  0.29313111305236816
Running inference for ./images/all_images/BA 2004-13069-15_Scan01.jpg... took:  0.2210540771484375
Running inference for ./images/all_images/BA 2004-06064_Scan01.jpg... took:  0.2559852600097656
Running inference for ./images/all_images/BA 2005-01260_Scan01.jpg... took:  0.20092105865478516
Running inference for ./images/all_images/BA 2004-13061-41_Scan01.jpg... took:  0.17557978630065918
Running inference for ./images/all_images/BA 2004-00036_Dok01.jpg... took:  0.21226286888122559
Running inference for ./images/all_images/BA 2004-01852_Scan01.jpg... 

KeyboardInterrupt: 